In [27]:
import pandas as pd
import datetime
import pytz
import plotly.graph_objects as go
from plotly.offline import iplot
from plotly.subplots import make_subplots
from scipy import stats

pd.set_option('display.max_rows', 400)
pd.set_option('display.max_columns', None)

icom_eia_forecasts = pd.read_csv('data/InvestingcomEIA.csv')
print("icom_eia_forecasts:")
display(icom_eia_forecasts)

fxstreet_eia_forecasts = pd.read_csv('data/FXStreetEIA.csv')
print("fxstreet_eia_forecasts:")
display(fxstreet_eia_forecasts)
print("fxstreet_api_forecasts:")
fxstreet_api_forecasts = pd.read_csv('data/FXStreetAPI.csv')
display(fxstreet_api_forecasts)

min_WTI = pd.read_csv(
    'data/cl-1m.csv',
    sep=';',
    header=None,
    names=['Date', 'Time', 'Open', 'High', 'Low', 'Close', 'Volume']
)

# Print the head of the DataFrame to verify it's loaded correctly
display(min_WTI)
min_WTI = min_WTI[1219109:]
print(min_WTI.loc[min_WTI['Date'] == '01/03/2011'])



icom_eia_forecasts:


,Release Date,Time,Actual,Forecast,Previous
0,12-Mar-25,8:30,NaN,NaN,3.614M
1,5-Mar-25,10:30,3.614M,0.600M,-2.332M
2,26-Feb-25,10:00,-2.332M,2.500M,4.633M
3,20-Feb-25,12:00,4.633M,3.200M,4.070M
4,12-Feb-25,10:30,4.070M,2.400M,8.664M
...,...,...,...,...,...
727,6-Apr-11,9:30,2.000M,1.300M,2.900M
728,30-Mar-11,9:30,2.900M,2.000M,2.100M
729,23-Mar-11,9:30,2.100M,2.000M,1.700M
730,16-Mar-11,9:30,1.700M,1.800M,2.500M


fxstreet_eia_forecasts:


,Date,Actual,Dev,Cons
0,03/05/2025 (Feb 28),3.614,1.19,-0.29
1,02/26/2025 (Feb 21),-2.332,-2.44,2.34
2,02/20/2025 (Feb 14),4.633,0.77,3.00
3,02/12/2025 (Feb 7),4.07,0.55,2.80
4,02/05/2025 (Jan 31),8.664,6.13,3.20
...,...,...,...,...
899,05/23/2007 (May 18),2,NaN,NaN
900,05/02/2007 (Apr 27),1,NaN,NaN
901,04/25/2007 (Apr 20),2.1,NaN,NaN
902,04/11/2007 (Apr 6),-1,NaN,NaN


fxstreet_api_forecasts:


,Date,Actual,Dev,Cons
0,03/04/2025 (Feb 28),-1.455,-0.36,-0.30
1,02/25/2025 (Feb 21),-0.640,NaN,NaN
2,02/19/2025 (Feb 14),3.340,0.36,2.20
3,02/11/2025 (Feb 7),9.043,2.64,2.80
4,02/04/2025 (Jan 31),5.025,0.79,3.17
...,...,...,...,...
500,8/4/2015,-2.400,NaN,NaN
501,7/28/2015,-1.900,NaN,NaN
502,7/21/2015,2.300,NaN,NaN
503,7/14/2015,-7.300,NaN,NaN


,Date,Time,Open,High,Low,Close,Volume
0,01/04/2007,17:00:00,66.03,66.31,66.03,66.10,197
1,01/04/2007,17:01:00,66.01,66.10,65.92,65.95,38
2,01/04/2007,17:02:00,65.97,66.08,65.96,66.05,64
3,01/04/2007,17:03:00,66.04,66.08,66.04,66.04,12
4,01/04/2007,17:04:00,66.09,66.10,66.03,66.04,17
...,...,...,...,...,...,...,...
6037529,11/03/2025,23:54:00,66.75,66.75,66.75,66.75,13
6037530,11/03/2025,23:56:00,66.75,66.75,66.74,66.74,3
6037531,11/03/2025,23:58:00,66.75,66.75,66.75,66.75,3
6037532,11/03/2025,23:59:00,66.75,66.75,66.75,66.75,1


               Date      Time    Open    High     Low   Close  Volume
1219109  01/03/2011  00:00:00   97.45   97.47   97.45   97.45      20
1219110  01/03/2011  00:01:00   97.46   97.46   97.46   97.46       2
1219111  01/03/2011  00:02:00   97.45   97.45   97.44   97.44       3
1219112  01/03/2011  00:04:00   97.45   97.45   97.45   97.45       5
1219113  01/03/2011  00:05:00   97.45   97.48   97.45   97.47      22
...             ...       ...     ...     ...     ...     ...     ...
1220478  01/03/2011  23:55:00  100.34  100.38  100.33  100.33       9
1220479  01/03/2011  23:56:00  100.34  100.34  100.33  100.33       7
1220480  01/03/2011  23:57:00  100.30  100.32  100.28  100.32      10
1220481  01/03/2011  23:58:00  100.32  100.32  100.32  100.32       1
1220482  01/03/2011  23:59:00  100.30  100.33  100.30  100.33       9

[1374 rows x 7 columns]


In [2]:
display(min_WTI)
display(icom_eia_forecasts)

,Date,Time,Open,High,Low,Close,Volume
0,01/04/2007,17:00:00,66.03,66.31,66.03,66.10,197
1,01/04/2007,17:01:00,66.01,66.10,65.92,65.95,38
2,01/04/2007,17:02:00,65.97,66.08,65.96,66.05,64
3,01/04/2007,17:03:00,66.04,66.08,66.04,66.04,12
4,01/04/2007,17:04:00,66.09,66.10,66.03,66.04,17
...,...,...,...,...,...,...,...
6037529,11/03/2025,23:54:00,66.75,66.75,66.75,66.75,13
6037530,11/03/2025,23:56:00,66.75,66.75,66.74,66.74,3
6037531,11/03/2025,23:58:00,66.75,66.75,66.75,66.75,3
6037532,11/03/2025,23:59:00,66.75,66.75,66.75,66.75,1


,Release Date,Time,Actual,Forecast,Previous
0,12-Mar-25,8:30,NaN,NaN,3.614M
1,5-Mar-25,10:30,3.614M,0.600M,-2.332M
2,26-Feb-25,10:00,-2.332M,2.500M,4.633M
3,20-Feb-25,12:00,4.633M,3.200M,4.070M
4,12-Feb-25,10:30,4.070M,2.400M,8.664M
...,...,...,...,...,...
727,6-Apr-11,9:30,2.000M,1.300M,2.900M
728,30-Mar-11,9:30,2.900M,2.000M,2.100M
729,23-Mar-11,9:30,2.100M,2.000M,1.700M
730,16-Mar-11,9:30,1.700M,1.800M,2.500M


In [3]:
def standardize_eia_datetime(row):
    release_date = pd.to_datetime(row['Release Date'], format='%d-%b-%y', errors='coerce').date()
    release_time = pd.to_datetime(row['Time'], format='%H:%M', errors='coerce').time()

    if pd.isna(release_date) or pd.isna(release_time):
        print('nan') #If this triggers, something is wrong
        return pd.NaT

    combined_datetime = pd.Timestamp.combine(release_date, release_time)
    return combined_datetime

icom_eia_forecasts['Release_Datetime'] = icom_eia_forecasts.apply(standardize_eia_datetime, axis=1)

display(icom_eia_forecasts)

eastern_tz = pytz.timezone('US/Eastern')
icom_eia_forecasts['Release_Datetime_EST'] = icom_eia_forecasts['Release_Datetime'].dt.tz_localize(eastern_tz, ambiguous='infer', nonexistent='shift_forward')

chicago_tz = pytz.timezone('America/Chicago')
icom_eia_forecasts['Release_Datetime_CST'] = icom_eia_forecasts['Release_Datetime_EST'].dt.tz_convert(chicago_tz)

display(icom_eia_forecasts)

,Release Date,Time,Actual,Forecast,Previous,Release_Datetime
0,12-Mar-25,8:30,NaN,NaN,3.614M,2025-03-12 08:30:00
1,5-Mar-25,10:30,3.614M,0.600M,-2.332M,2025-03-05 10:30:00
2,26-Feb-25,10:00,-2.332M,2.500M,4.633M,2025-02-26 10:00:00
3,20-Feb-25,12:00,4.633M,3.200M,4.070M,2025-02-20 12:00:00
4,12-Feb-25,10:30,4.070M,2.400M,8.664M,2025-02-12 10:30:00
...,...,...,...,...,...,...
727,6-Apr-11,9:30,2.000M,1.300M,2.900M,2011-04-06 09:30:00
728,30-Mar-11,9:30,2.900M,2.000M,2.100M,2011-03-30 09:30:00
729,23-Mar-11,9:30,2.100M,2.000M,1.700M,2011-03-23 09:30:00
730,16-Mar-11,9:30,1.700M,1.800M,2.500M,2011-03-16 09:30:00


,Release Date,Time,Actual,Forecast,Previous,Release_Datetime,Release_Datetime_EST,Release_Datetime_CST
0,12-Mar-25,8:30,NaN,NaN,3.614M,2025-03-12 08:30:00,2025-03-12 08:30:00-04:00,2025-03-12 07:30:00-05:00
1,5-Mar-25,10:30,3.614M,0.600M,-2.332M,2025-03-05 10:30:00,2025-03-05 10:30:00-05:00,2025-03-05 09:30:00-06:00
2,26-Feb-25,10:00,-2.332M,2.500M,4.633M,2025-02-26 10:00:00,2025-02-26 10:00:00-05:00,2025-02-26 09:00:00-06:00
3,20-Feb-25,12:00,4.633M,3.200M,4.070M,2025-02-20 12:00:00,2025-02-20 12:00:00-05:00,2025-02-20 11:00:00-06:00
4,12-Feb-25,10:30,4.070M,2.400M,8.664M,2025-02-12 10:30:00,2025-02-12 10:30:00-05:00,2025-02-12 09:30:00-06:00
...,...,...,...,...,...,...,...,...
727,6-Apr-11,9:30,2.000M,1.300M,2.900M,2011-04-06 09:30:00,2011-04-06 09:30:00-04:00,2011-04-06 08:30:00-05:00
728,30-Mar-11,9:30,2.900M,2.000M,2.100M,2011-03-30 09:30:00,2011-03-30 09:30:00-04:00,2011-03-30 08:30:00-05:00
729,23-Mar-11,9:30,2.100M,2.000M,1.700M,2011-03-23 09:30:00,2011-03-23 09:30:00-04:00,2011-03-23 08:30:00-05:00
730,16-Mar-11,9:30,1.700M,1.800M,2.500M,2011-03-16 09:30:00,2011-03-16 09:30:00-04:00,2011-03-16 08:30:00-05:00


In [4]:
def standardize_wti_datetime(row):
    """Standardizes datetime for min_res_OIH dataframe."""
    date_str = row['Date']
    time_str = row['Time']

    #print(date_str + ' ' + time_str)#debug

    combined_datetime = pd.to_datetime(date_str + ' ' + time_str, format='%d/%m/%Y %H:%M:%S', errors='coerce') # Combined format

    #print(combined_datetime)#debug

    if pd.isna(combined_datetime):
        print('nan')#If this triggers, something is wrong
        return pd.NaT
    return combined_datetime

min_WTI['Datetime'] = min_WTI.apply(standardize_wti_datetime, axis=1)

min_WTI['Release_Datetime_CST'] = min_WTI['Datetime'].dt.tz_localize(chicago_tz, ambiguous='infer', nonexistent='shift_forward')
display(min_WTI)

,Date,Time,Open,High,Low,Close,Volume,Datetime,Release_Datetime_CST
0,01/04/2007,17:00:00,66.03,66.31,66.03,66.10,197,2007-04-01 17:00:00,2007-04-01 17:00:00-05:00
1,01/04/2007,17:01:00,66.01,66.10,65.92,65.95,38,2007-04-01 17:01:00,2007-04-01 17:01:00-05:00
2,01/04/2007,17:02:00,65.97,66.08,65.96,66.05,64,2007-04-01 17:02:00,2007-04-01 17:02:00-05:00
3,01/04/2007,17:03:00,66.04,66.08,66.04,66.04,12,2007-04-01 17:03:00,2007-04-01 17:03:00-05:00
4,01/04/2007,17:04:00,66.09,66.10,66.03,66.04,17,2007-04-01 17:04:00,2007-04-01 17:04:00-05:00
...,...,...,...,...,...,...,...,...,...
6037529,11/03/2025,23:54:00,66.75,66.75,66.75,66.75,13,2025-03-11 23:54:00,2025-03-11 23:54:00-05:00
6037530,11/03/2025,23:56:00,66.75,66.75,66.74,66.74,3,2025-03-11 23:56:00,2025-03-11 23:56:00-05:00
6037531,11/03/2025,23:58:00,66.75,66.75,66.75,66.75,3,2025-03-11 23:58:00,2025-03-11 23:58:00-05:00
6037532,11/03/2025,23:59:00,66.75,66.75,66.75,66.75,1,2025-03-11 23:59:00,2025-03-11 23:59:00-05:00


In [5]:
icom_eia_forecasts = icom_eia_forecasts.set_index('Release_Datetime_CST').sort_index()
min_WTI = min_WTI.set_index('Release_Datetime_CST').sort_index()

In [ ]:
display(icom_eia_forecasts)
display(min_WTI)


,Release Date,Time,Actual,Forecast,Previous,Release_Datetime,Release_Datetime_EST,supply_surprise
Release_Datetime_CST,,,,,,,,
2011-03-09 09:30:00-06:00,9-Mar-11,10:30,2.500M,1.100M,-0.400M,2011-03-09 10:30:00,2011-03-09 10:30:00-05:00,1.400
2011-03-16 08:30:00-05:00,16-Mar-11,9:30,1.700M,1.800M,2.500M,2011-03-16 09:30:00,2011-03-16 09:30:00-04:00,-0.100
2011-03-23 08:30:00-05:00,23-Mar-11,9:30,2.100M,2.000M,1.700M,2011-03-23 09:30:00,2011-03-23 09:30:00-04:00,0.100
2011-03-30 08:30:00-05:00,30-Mar-11,9:30,2.900M,2.000M,2.100M,2011-03-30 09:30:00,2011-03-30 09:30:00-04:00,0.900
2011-04-06 08:30:00-05:00,6-Apr-11,9:30,2.000M,1.300M,2.900M,2011-04-06 09:30:00,2011-04-06 09:30:00-04:00,0.700
...,...,...,...,...,...,...,...,...
2025-02-12 09:30:00-06:00,12-Feb-25,10:30,4.070M,2.400M,8.664M,2025-02-12 10:30:00,2025-02-12 10:30:00-05:00,1.670
2025-02-20 11:00:00-06:00,20-Feb-25,12:00,4.633M,3.200M,4.070M,2025-02-20 12:00:00,2025-02-20 12:00:00-05:00,1.433
2025-02-26 09:00:00-06:00,26-Feb-25,10:00,-2.332M,2.500M,4.633M,2025-02-26 10:00:00,2025-02-26 10:00:00-05:00,-4.832


,Date,Time,Open,High,Low,Close,Volume,Datetime
Release_Datetime_CST,,,,,,,,
2007-04-01 17:00:00-05:00,01/04/2007,17:00:00,66.03,66.31,66.03,66.10,197,2007-04-01 17:00:00
2007-04-01 17:01:00-05:00,01/04/2007,17:01:00,66.01,66.10,65.92,65.95,38,2007-04-01 17:01:00
2007-04-01 17:02:00-05:00,01/04/2007,17:02:00,65.97,66.08,65.96,66.05,64,2007-04-01 17:02:00
2007-04-01 17:03:00-05:00,01/04/2007,17:03:00,66.04,66.08,66.04,66.04,12,2007-04-01 17:03:00
2007-04-01 17:04:00-05:00,01/04/2007,17:04:00,66.09,66.10,66.03,66.04,17,2007-04-01 17:04:00
...,...,...,...,...,...,...,...,...
2025-03-11 23:54:00-05:00,11/03/2025,23:54:00,66.75,66.75,66.75,66.75,13,2025-03-11 23:54:00
2025-03-11 23:56:00-05:00,11/03/2025,23:56:00,66.75,66.75,66.74,66.74,3,2025-03-11 23:56:00
2025-03-11 23:58:00-05:00,11/03/2025,23:58:00,66.75,66.75,66.75,66.75,3,2025-03-11 23:58:00


KeyError: 'False: boolean label can not be used without a boolean index'

In [ ]:
'''
def calculate_price_change(eia_release_times, price_data, time_interval_minutes):
    """
    Calculates the percentage price change in the price_data dataframe
    for each release time in eia_release_times, over the specified time interval.

    Args:
        eia_release_times (pd.DatetimeIndex): Index of the icom_eia_forecasts dataframe (release datetimes).
        price_data (pd.DataFrame): min_res_OIH dataframe with Datetime index and 'Close' price column.
        time_interval_minutes (int): Time interval in minutes for price change calculation.

    Returns:
        list: List of price changes for each release event, or pd.NA if data is missing.
    """
    price_changes = []
    for release_time in eia_release_times:
        try:
            # Get price at release time
            release_price = price_data.loc[release_time]['Close']

            # Get price at time_interval_minutes after release
            future_time = release_time + pd.Timedelta(minutes=time_interval_minutes)
            future_price = price_data.loc[future_time]['Close']

            price_change = ((future_price - release_price) / release_price) * 100.0
            price_changes.append(price_change)

        except KeyError:
            #price data is missing when 0 volume so give 0 % price change
            price_changes.append(0.0)

    return price_changes

time_intervals_minutes = [1, 5, 10, 60]

price_change_column_names = []
for interval in time_intervals_minutes:
    col_name = f'{interval}_Minute_Percent_Price_Change' if interval < 60 else f'{interval//60}_Hour_Percent_Price_Change'
    price_change_column_names.append(col_name)

# remove cols if already there
columns_to_drop = [col for col in price_change_column_names if col in icom_eia_forecasts.columns]
if columns_to_drop:
    print(f"Removing existing columns: {columns_to_drop}")
    icom_eia_forecasts.drop(columns=columns_to_drop, inplace=True)

# add % price change columns
for interval, col_name in zip(time_intervals_minutes, price_change_column_names):
    icom_eia_forecasts[col_name] = calculate_price_change(icom_eia_forecasts.index, min_res_OIH, interval)


display(icom_eia_forecasts)
'''

In [ ]:
'''

percentage_change_columns = [
    '1_Minute_Percent_Price_Change',
    '5_Minute_Percent_Price_Change',
    '10_Minute_Percent_Price_Change',
    '1_Hour_Percent_Price_Change'
]

average_price_changes = icom_eia_forecasts[percentage_change_columns].abs().mean() # want to look at raw magnitude of price change values, so abs()

intervals = [
    '1-Minute',
    '5-Minute',
    '10-Minute',
    '1-Hour'
]


fig = go.Figure(data=[go.Bar(x=intervals, y=average_price_changes.values,
                            text=average_price_changes.values.round(3), # Display values on bars
                            textposition='outside')])

fig.update_layout(
    title='Average Percentage Price Change Following EIA WPSR Report Releases',
    xaxis_title='Time Interval After Release',
    yaxis_title='Average Percentage Price Change',
    yaxis_gridcolor='lightgray' # Add light gray grid lines for better readability
)

iplot(fig)
'''

In [7]:
def calculate_supply_surprise(df):
    """
    Calculates the 'supply_surprise' column as the difference between 'Actual' and 'Forecast'.
    Handles 'M' suffix and converts to numeric.

    Args:
        df (pd.DataFrame): icom_eia_forecasts DataFrame.

    Returns:
        pd.Series: Supply surprise values.
    """
    actual = df['Actual'].str.replace('M', '', regex=False).astype(float)
    forecast = df['Forecast'].str.replace('M', '', regex=False).astype(float)
    supply_surprise = actual - forecast
    return supply_surprise

icom_eia_forecasts['supply_surprise'] = calculate_supply_surprise(icom_eia_forecasts)

display(icom_eia_forecasts)

,Release Date,Time,Actual,Forecast,Previous,Release_Datetime,Release_Datetime_EST,supply_surprise
Release_Datetime_CST,,,,,,,,
2011-03-09 09:30:00-06:00,9-Mar-11,10:30,2.500M,1.100M,-0.400M,2011-03-09 10:30:00,2011-03-09 10:30:00-05:00,1.400
2011-03-16 08:30:00-05:00,16-Mar-11,9:30,1.700M,1.800M,2.500M,2011-03-16 09:30:00,2011-03-16 09:30:00-04:00,-0.100
2011-03-23 08:30:00-05:00,23-Mar-11,9:30,2.100M,2.000M,1.700M,2011-03-23 09:30:00,2011-03-23 09:30:00-04:00,0.100
2011-03-30 08:30:00-05:00,30-Mar-11,9:30,2.900M,2.000M,2.100M,2011-03-30 09:30:00,2011-03-30 09:30:00-04:00,0.900
2011-04-06 08:30:00-05:00,6-Apr-11,9:30,2.000M,1.300M,2.900M,2011-04-06 09:30:00,2011-04-06 09:30:00-04:00,0.700
...,...,...,...,...,...,...,...,...
2025-02-12 09:30:00-06:00,12-Feb-25,10:30,4.070M,2.400M,8.664M,2025-02-12 10:30:00,2025-02-12 10:30:00-05:00,1.670
2025-02-20 11:00:00-06:00,20-Feb-25,12:00,4.633M,3.200M,4.070M,2025-02-20 12:00:00,2025-02-20 12:00:00-05:00,1.433
2025-02-26 09:00:00-06:00,26-Feb-25,10:00,-2.332M,2.500M,4.633M,2025-02-26 10:00:00,2025-02-26 10:00:00-05:00,-4.832


In [ ]:
supply_surprise_data = icom_eia_forecasts['supply_surprise']

fig_hist = go.Figure(data=[go.Histogram(x=supply_surprise_data, nbinsx=100)])

fig_hist.update_layout(
    title='Histogram of EIA WPSR Crude Oil Supply Surprise',
    xaxis_title='Supply Surprise (Actual - Forecast, Million Barrels)',
    yaxis_title='Frequency (Number of Releases)',
    bargap=0.1
)

fig_hist.show()

In [ ]:
'''
price_change_columns = [
    '1_Minute_Percent_Price_Change',
    '5_Minute_Percent_Price_Change',
    '10_Minute_Percent_Price_Change',
    '1_Hour_Percent_Price_Change'
]

for col in price_change_columns:
    price_change_data = icom_eia_forecasts[col].dropna()

    fig_hist_price_change = go.Figure(data=[go.Histogram(x=price_change_data, nbinsx=100)])

    fig_hist_price_change.update_layout(
        title=f'Histogram of {col} After EIA WPSR Release',
        xaxis_title='Percentage Price Change (%)',
        yaxis_title='Frequency (Number of Releases)',
        bargap=0.1
    )

    fig_hist_price_change.show()
'''

In [ ]:
'''
from plotly.subplots import make_subplots
from scipy import stats  # Import scipy.stats for statistical tests

# Assume your dataframes are already loaded as icom_eia_forecasts and min_res_OIH
# and that you have run all the previous code blocks, including price change calculations and supply_surprise

# --- Kolmogorov-Smirnov Test ---

price_change_columns = [
    '1_Minute_Percent_Price_Change',
    '5_Minute_Percent_Price_Change',
    '10_Minute_Percent_Price_Change',
    '1_Hour_Percent_Price_Change'
]

print("Kolmogorov-Smirnov Test for Normality of Price Changes:\n")

for col in price_change_columns:
    price_change_data = icom_eia_forecasts[col].dropna() # Drop NA values

    # Standardize the data to have mean=0 and std=1 for comparison with standard normal
    standardized_data = (price_change_data - price_change_data.mean()) / price_change_data.std()

    # Perform Kolmogorov-Smirnov test against standard normal distribution (cdf='norm')
    ks_statistic, p_value = stats.kstest(standardized_data, 'norm')

    print(f"Column: {col}")
    print(f"KS Statistic: {ks_statistic:.4f}")
    print(f"P-value: {p_value:.4f}")

    alpha = 0.05  # Significance level
    if p_value < alpha:
        print(f"P-value < {alpha}: Reject the null hypothesis. Distribution is significantly different from normal.\n")
    else:
        print(f"P-value >= {alpha}: Fail to reject the null hypothesis. Distribution is not significantly different from normal (at alpha={alpha}).\n")
    print("-" * 40)
'''

In [ ]:
#34.3 seconds

percentage_price_changes_1min_wti = []
previous_close_price = None

for index, row in min_WTI.iterrows():
    current_close_price = row['Close']
    if previous_close_price is not None and previous_close_price != 0:
        percentage_change = ((current_close_price - previous_close_price) / previous_close_price) * 100.0
        percentage_price_changes_1min_wti.append(percentage_change)
    previous_close_price = current_close_price

percentage_price_changes_1min_wti_series = pd.Series(percentage_price_changes_1min_wti)
percentage_price_changes_1min_wti_series = percentage_price_changes_1min_wti_series.dropna()

# --- Histogram of Filtered One-Minute Percentage Price Changes in min_res_OIH ---

fig_hist_1min_wti = go.Figure(data=[go.Histogram(x=percentage_price_changes_1min_wti)])

fig_hist_1min_wti.update_layout(
    title='Histogram of One-Minute Percentage Price Changes in Crude Price',
    xaxis_title='One-Minute Percentage Price Change (%)',
    yaxis_title='Frequency (Number of Minutes)',
    xaxis_range=[-0.5, 0.5],
    yaxis_range=[0, 300000]
)

fig_hist_1min_wti.show()


In [20]:
min_WTI.describe()#.apply(lambda s: s.apply('{0:.5f}'.format))

,Open,High,Low,Close,Volume,Datetime
count,6.037534e+06,6.037534e+06,6.037534e+06,6.037534e+06,6.037534e+06,6037534
mean,7.282282e+01,7.284607e+01,7.279951e+01,7.282280e+01,1.999807e+02,2016-06-08 09:56:21.748896
min,6.580000e+00,7.540000e+00,6.500000e+00,6.900000e+00,0.000000e+00,2007-04-01 17:00:00
25%,5.408000e+01,5.410000e+01,5.407000e+01,5.408000e+01,1.500000e+01,2012-01-12 10:30:15
50%,7.281000e+01,7.283000e+01,7.279000e+01,7.281000e+01,6.200000e+01,2016-07-11 01:30:30
75%,8.958000e+01,8.961000e+01,8.956000e+01,8.959000e+01,2.190000e+02,2020-11-10 06:15:45
max,1.470900e+02,1.472700e+02,1.470000e+02,1.470800e+02,3.077500e+04,2025-03-12 00:00:00
std,2.190417e+01,2.190779e+01,2.190053e+01,2.190417e+01,4.027235e+02,NaN


In [ ]:
'''
price_change_columns_eia = [
    '1_Minute_Percent_Price_Change',
    '5_Minute_Percent_Price_Change',
    '10_Minute_Percent_Price_Change',
    '1_Hour_Percent_Price_Change'
]

print("Kolmogorov-Smirnov Test Comparing Distributions:\n"
      "  Distribution 1: One-Minute Percentage Price Changes (All Minutes)\n"
      "  Distribution 2: Percentage Price Changes After EIA WPSR Release (at different intervals)\n\n")

for col_eia in price_change_columns_eia:
    price_change_data_eia = icom_eia_forecasts[col_eia].dropna() # Drop NA values from EIA release price changes

    # Ensure both datasets have data to compare, if not skip this column
    if price_change_data_eia.empty or percentage_price_changes_1min_oih_series.empty:
        print(f"Warning: Skipping KS test for {col_eia} due to empty data in one or both samples.")
        print("-" * 40)
        continue


    # Perform 2-sample Kolmogorov-Smirnov test to compare EIA release price changes vs. all 1-minute price changes
    ks_statistic_2samp, p_value_2samp = stats.ks_2samp(price_change_data_eia, percentage_price_changes_1min_oih_series)

    print(f"EIA Release Price Change Column: {col_eia}")
    print(f"KS Statistic (2-sample): {ks_statistic_2samp:.4f}")
    print(f"P-value (2-sample): {p_value_2samp:.4f}")

    alpha = 0.05  # Significance level
    if p_value_2samp < alpha:
        print(f"P-value < {alpha}: Reject the null hypothesis. Distributions are significantly different.\n")
    else:
        print(f"P-value >= {alpha}: Fail to reject the null hypothesis. Distributions are not significantly different (at alpha={alpha}).\n")
    print("-" * 40)
'''

In [ ]:
def get_price_windows(eia_release_times, price_data, window_minutes_before=60, window_minutes_after=60):
    """
    Extracts price data windows around EIA report release times.

    Args:
        eia_release_times (pd.DatetimeIndex): Index of icom_eia_forecasts (release datetimes).
        price_data (pd.DataFrame): min_res_OIH dataframe with Datetime index.
        window_minutes_before (int): Minutes to include before release time.
        window_minutes_after (int): Minutes to include after release time.

    Returns:
        pd.DataFrame: A DataFrame containing price data for all events, within the specified windows.
                      Returns an empty DataFrame if no data is found within any window.
    """
    price_windows_list = []

    for release_time in eia_release_times:
        start_time = release_time - pd.Timedelta(minutes=window_minutes_before)
        print(start_time)
        end_time = release_time + pd.Timedelta(minutes=window_minutes_after)
        print(end_time)

        # Select rows from min_res_OIH within the time window
        window_data = price_data.loc[start_time:end_time].copy()
        #display(window_data.head)

        if not window_data.empty: # Only append if there is data in the window
            window_data['Release_Datetime'] = release_time # Add release time as a column for reference
            price_windows_list.append(window_data)

    if price_windows_list:
        price_windows_df = pd.concat(price_windows_list)
        return price_windows_df
    else:
        return pd.DataFrame() # Return empty DataFrame if no data found in any window


# --- Extract Price Windows ---
price_window_60min = get_price_windows(icom_eia_forecasts.index, min_re, window_minutes_before=60, window_minutes_after=60)
price_window_60min.sort_index(inplace=True)
if not price_window_60min.empty:
    print("Price data windows extracted successfully!")
    print("price_window_60min.head():")
    display(price_window_60min.head(240))
    print("\nprice_window_60min.tail():")
    display(price_window_60min.tail(240))
    print(f"\nShape of price_window_60min: {price_window_60min.shape}")
else:
    print("No price data found within the specified windows around EIA releases.")

In [ ]:
display(icom_eia_forecasts)
display(min_res_OIH)
display(price_window_60min)

In [ ]:

# Assume your price_window_60min dataframe is already loaded
# and has columns 'Datetime', 'Release_Datetime', and 'Volume'

# --- Calculate Time to Release (in minutes) ---
price_window_60min['Time_to_Release_Minutes'] = (price_window_60min.index - price_window_60min['Release_Datetime']).dt.total_seconds() / 60
display(price_window_60min.head(240))
# --- Group by Time to Release and Calculate Average Volume ---
average_volume_by_time = price_window_60min.groupby('Time_to_Release_Minutes')['Volume'].mean().reset_index()

# --- Create Scatter Plot of Average Volume ---
fig_volume_scatter = go.Figure(data=[go.Scatter(
    x=average_volume_by_time['Time_to_Release_Minutes'],
    y=average_volume_by_time['Volume'],
    mode='markers', # Use markers for scatter plot
    marker=dict(size=8),
    text=average_volume_by_time['Volume'], # Hover text is the volume
    hovertemplate="Time to Release: %{x:.0f} minutes<br>Average Volume: %{y:.0f}<extra></extra>" # Custom hover template
)])

fig_volume_scatter.update_layout(

    title='Average Trading Volume Around EIA WPSR Release',
    xaxis_title='Minutes Relative to EIA Report Release',
    yaxis_title='Average Volume',
    xaxis=dict(
        tickvals=[-60, -45, -30, -15, 0, 15, 30, 45, 60], # Define specific x-axis ticks
        ticktext=['-60', '-45', '-30', '-15', 'Release', '+15', '+30', '+45', '+60'] # Custom tick labels
    ),
    hovermode="closest" # настроить hovermode для лучшего взаимодействия
)

fig_volume_scatter.show()

In [ ]:

import numpy as np
# Assume price_window_60min dataframe is already loaded and has 'Datetime', 'Close', and 'Release_Datetime' columns

# 1. Calculate Price Changes Relative to Pre-Release Minute
price_window_60min['Time_Delta_Minutes'] = price_window_60min.index - price_window_60min['Release_Datetime']
price_window_60min['Time_Delta_Minutes'] = price_window_60min['Time_Delta_Minutes'].apply(lambda td: td.total_seconds() / 60)
price_window_60min['Price_Change'] = price_window_60min.groupby('Release_Datetime')['Close'].transform(lambda x: x - x.shift(1))
price_window_60min['Percent_Change'] = price_window_60min.groupby('Release_Datetime')['Close'].pct_change() * 100

price_window_60min = price_window_60min.dropna(subset=['Percent_Change']) # Remove rows where shift resulted in NaN

# 2. Define Time Intervals and Price Change Bins for Heatmap
time_intervals = sorted(price_window_60min['Time_Delta_Minutes'].unique())
price_change_bins = np.linspace(-1, 1, num=41) # Example: Bins from -1% to +1% in 0.05% increments, adjust as needed
price_change_labels = [f'{bin_val:.2f}%' for bin_val in price_change_bins] # Labels for bins

# 3.  Bin Price Changes and Count Frequencies
price_window_60min['Price_Change_Bin'] = pd.cut(price_window_60min['Percent_Change'], bins=price_change_bins, labels=price_change_labels[:-1], include_lowest=True)

display(price_window_60min)

heatmap_data = price_window_60min.groupby(['Price_Change_Bin', 'Time_Delta_Minutes']).size().unstack(fill_value=0)

# Reindex columns and rows to ensure all intervals and bins are present, even if count is zero
heatmap_data = heatmap_data.reindex(columns=time_intervals, fill_value=0)
heatmap_data = heatmap_data.reindex(index=price_change_labels[:-1], fill_value=0)
display(heatmap_data)

# 4. Create Interactive Heatmap with Plotly
fig_heatmap = go.Figure(data=go.Heatmap(
    z=heatmap_data.values,
    x=heatmap_data.columns,
    y=heatmap_data.index,
    colorscale='Viridis', # Choose a colorscale that suits your preference
    colorbar=dict(title='Frequency')
))

fig_heatmap.update_layout(
    title='Frequency of Percentage Price Changes Around EIA WPSR Release',
    xaxis_title='Time Relative to Release (Minutes)',
    yaxis_title='Percentage Price Change Bins',
    yaxis=dict(autorange="reversed"), # Invert y-axis to have positive changes at the top
    xaxis=dict(tickvals=time_intervals[::5], ticktext=[int(x) for x in time_intervals[::5]]) # Show x-axis ticks every 5 minutes
)

fig_heatmap.show()

In [ ]:
#fourier transform for max frequency
# --- Group by Time to Release and Calculate Average Volume ---
average_volume_by_time = price_window_60min.groupby('Time_Delta_Minutes')['Volume'].mean().reset_index()

# --- Create Scatter Plot of Average Volume ---
fig_volume_scatter = go.Figure(data=[go.Scatter(
    x=average_volume_by_time['Time_Delta_Minutes'],
    y=average_volume_by_time['Volume'],
    mode='markers', # Use markers for scatter plot
    marker=dict(size=8),
    text=average_volume_by_time['Volume'], # Hover text is the volume
    hovertemplate="Time to Release: %{x:.0f} minutes<br>Average Volume: %{y:.0f}<extra></extra>" # Custom hover template
)])

fig_volume_scatter.update_layout(
    title='Average Trading Volume Around EIA WPSR Release',
    xaxis_title='Minutes Relative to EIA Report Release',
    yaxis_title='Average Volume',
    xaxis=dict(
        tickvals=[-60, -45, -30, -15, 0, 15, 30, 45, 60], # Define specific x-axis ticks
        ticktext=['-60', '-45', '-30', '-15', 'Release', '+15', '+30', '+45', '+60'] # Custom tick labels
    ),
    hovermode="closest"
)

fig_volume_scatter.show()

In [ ]:


# Assume your dataframes are already loaded as icom_eia_forecasts and min_res_OIH
# and that you have run the datetime standardization in previous steps if necessary

# 1. Extract EIA Release Dates from icom_eia_forecasts
eia_release_dates_str = icom_eia_forecasts['Release Date'].unique()

# Convert EIA release dates to datetime objects for consistent comparison
eia_release_dates_datetime = pd.to_datetime(eia_release_dates_str, format='%d-%b-%y', errors='coerce')

# Drop any potential NaT values if date conversion failed for any release dates
eia_release_dates_datetime = eia_release_dates_datetime.dropna()

# 2. Convert 'Date' column in min_res_OIH to datetime objects
min_res_OIH['Date_Datetime'] = pd.to_datetime(min_res_OIH['Date'], format='%m/%d/%Y', errors='coerce')

# Drop rows where 'Date' conversion failed (if any) - optional, but good practice
min_res_OIH = min_res_OIH.dropna(subset=['Date_Datetime'])

# 3. Filter min_res_OIH to exclude EIA release days
min_res_OIH_no_eia_days = min_res_OIH[~min_res_OIH['Date_Datetime'].isin(eia_release_dates_datetime)]

# Drop the temporary 'Date_Datetime' column if you don't need it anymore
min_res_OIH_no_eia_days = min_res_OIH_no_eia_days.drop(columns=['Date_Datetime'])

# Reset index if needed
min_res_OIH_no_eia_days = min_res_OIH_no_eia_days.reset_index(drop=True)

# Display the head of the new dataframe to confirm
print(min_res_OIH_no_eia_days.head())

# Optionally, check the shape difference to see how many days were removed
print(f"\nOriginal min_res_OIH shape: {min_res_OIH.shape}")
print(f"Shape of min_res_OIH without EIA days: {min_res_OIH_no_eia_days.shape}")

In [ ]:
print(f"\nOriginal min_res_OIH shape: {min_res_OIH.shape}")
print(f"Shape of min_res_OIH without EIA days: {min_res_OIH_no_eia_days.shape}")

# Assume your min_res_OIH dataframe is already loaded

# --- 1. Extract Time of Day ---
# Convert 'Time' column to datetime.time objects
min_res_OIH['Time_of_Day'] = pd.to_datetime(min_res_OIH['Time'], format='%H:%M', errors='coerce').dt.time

# Drop rows where Time_of_Day conversion failed (if any)
min_res_OIH = min_res_OIH.dropna(subset=['Time_of_Day'])

# --- 2. Group by Time of Day and 3. Calculate Average Volume ---
average_volume_intraday = min_res_OIH.groupby('Time_of_Day')['Volume'].mean().reset_index()

# --- 4. Prepare for Plotting ---
time_labels = [time_obj.strftime('%H:%M') for time_obj in average_volume_intraday['Time_of_Day']] # Format time for labels

# --- 5. Create Line Plot ---
fig_intraday_volume = go.Figure(data=[go.Scatter(
    x=time_labels,
    y=average_volume_intraday['Volume'],
    mode='lines', # Use lines to show trend
    name='Average Volume',
    line=dict(color='blue')
)])

fig_intraday_volume.update_layout(
    title='Average Intraday Trading Volume for OIH',
    xaxis_title='Time of Day (Eastern Time)',
    yaxis_title='Average Volume',
    xaxis=dict(
        tickangle=-45,
        tickmode='array',
        tickvals=time_labels[::30],  # Show every 30 minutes for readability - adjust as needed
        ticktext=time_labels[::30]
    ),
    hovermode="closest"
)

fig_intraday_volume.show()

In [ ]:
print(f"\nOriginal min_res_OIH shape: {min_res_OIH.shape}")
print(f"Shape of min_res_OIH without EIA days: {min_res_OIH_no_eia_days.shape}")

# Assume your min_res_OIH dataframe is already loaded

# --- 1. Extract Time of Day ---
# Convert 'Time' column to datetime.time objects
min_res_OIH_no_eia_days['Time_of_Day'] = pd.to_datetime(min_res_OIH_no_eia_days['Time'], format='%H:%M', errors='coerce').dt.time

# Drop rows where Time_of_Day conversion failed (if any)
min_res_OIH = min_res_OIH_no_eia_days.dropna(subset=['Time_of_Day'])

# --- 2. Group by Time of Day and 3. Calculate Average Volume ---
average_volume_intraday = min_res_OIH_no_eia_days.groupby('Time_of_Day')['Volume'].mean().reset_index()

# --- 4. Prepare for Plotting ---
time_labels = [time_obj.strftime('%H:%M') for time_obj in average_volume_intraday['Time_of_Day']] # Format time for labels

# --- 5. Create Line Plot ---
fig_intraday_volume = go.Figure(data=[go.Scatter(
    x=time_labels,
    y=average_volume_intraday['Volume'],
    mode='lines', # Use lines to show trend
    name='Average Volume',
    line=dict(color='blue')
)])

fig_intraday_volume.update_layout(
    title='Average Intraday Trading Volume for OIH',
    xaxis_title='Time of Day (Eastern Time)',
    yaxis_title='Average Volume',
    xaxis=dict(
        tickangle=-45,
        tickmode='array',
        tickvals=time_labels[::30],  # Show every 30 minutes for readability - adjust as needed
        ticktext=time_labels[::30]
    ),
    hovermode="closest"
)

fig_intraday_volume.show()